# Класифікує лог-файли Kubernetes (namespace/node/pod) на основі перших рядків.
Після цього створюється словник `file_map` з відповідним розподілом файлів.


In [1]:
import os
import re
import pandas as pd

def classify_kubectl_file(filepath):
    with open(filepath, "r", encoding="utf-8") as f:
        for _ in range(10):
            line = f.readline().lower()
            if "kubectl get pods" in line:
                return "namespace"
            elif "kubectl describe node" in line:
                return "node"
            elif "kubectl describe pod" in line:
                return "pod"
    return "unknown"

def load_all_logs(directory="."):
    file_map = {
        "namespace": None,
        "nodes": [],
        "pods": [],
        "unknown": []
    }

    for fname in os.listdir(directory):
        full_path = os.path.join(directory, fname)
        if not os.path.isfile(full_path):
            continue
        file_type = classify_kubectl_file(full_path)
        if file_type == "namespace":
            file_map["namespace"] = full_path
        elif file_type == "node":
            file_map["nodes"].append(full_path)
        elif file_type == "pod":
            file_map["pods"].append(full_path)
        else:
            file_map["unknown"].append(full_path)

    return file_map

log_dir = "all_logs"
file_map = load_all_logs(log_dir)

print("Класифікація завершена:")
print(f"Namespace-файл: {file_map['namespace']}")
print(f"Node-файлів: {len(file_map['nodes'])}")
print(f"Pod-файлів: {len(file_map['pods'])}")
print(f"Unknown-файлів: {file_map['unknown']}")


Класифікація завершена:
Namespace-файл: all_logs\01_namespace_sample.log
Node-файлів: 2
Pod-файлів: 2
Unknown-файлів: []


# Парсить вивід `kubectl get pods` для аналізу статусів подів та їх розміщення на нодах.
Повертає DataFrame з інформацією про pod-и.

In [2]:
def analyze_namespace(filepath):
    print(f"\nАналіз namespace з файлу: {filepath}")

    with open(filepath, "r", encoding="utf-8") as f:
        lines = [line.strip() for line in f if line.strip()]

    # пошук строки заголовку
    header_line = next((line for line in lines if line.startswith("NAME")), None)
    if not header_line:
        print("Заголовок не знайдено.")
        return None

    headers = re.split(r'\s{2,}', header_line)
    header_index = lines.index(header_line)
    data_lines = lines[header_index + 1:]

    parsed_rows = []
    for row in data_lines:
        row_values = re.split(r'\s{2,}', row)
        if len(row_values) < len(headers):
            row_values += [None] * (len(headers) - len(row_values))
        parsed_rows.append(row_values)

    df = pd.DataFrame(parsed_rows, columns=headers)
    df["NODE"] = df["NODE"].fillna("Unknown")

    print(f"Завантажено {len(df)} pod-ів")
    print("\nСтатус подів:")
    print(df["STATUS"].value_counts())

    print("\nПоди по нодах:")
    print(df["NODE"].value_counts())

    return df


In [3]:
df_namespace = analyze_namespace(file_map["namespace"])



Аналіз namespace з файлу: all_logs\01_namespace_sample.log
Завантажено 10 pod-ів

Статус подів:
STATUS
Completed    5
Running      5
Name: count, dtype: int64

Поди по нодах:
NODE
ip-100-100-24-129.eu-west-1.compute.internal    4
ip-100-100-46-4.eu-west-1.compute.internal      1
ip-100-100-31-113.eu-west-1.compute.internal    1
ip-100-100-35-194.eu-west-1.compute.internal    1
ip-100-100-41-106.eu-west-1.compute.internal    1
ip-100-100-58-211.eu-west-1.compute.internal    1
ip-100-100-17-161.eu-west-1.compute.internal    1
Name: count, dtype: int64


# Аналізує вивід `kubectl describe node`: витягує allocatable ресурси, перевіряє перевантаження,
обчислює загальне використання CPU та RAM, виявляє бази даних і поди без resource requests/limits.

Також формує DataFrame з детальною інформацією про pod-и.


In [4]:
import os
import re
import pandas as pd

# CPU/Memory парсери
def parse_cpu(cpu_str: str) -> float:
    try:
        if not cpu_str or not isinstance(cpu_str, str):
            return 0.0
        cpu_str = cpu_str.strip().lower().split()[0].split('(')[0]
        if cpu_str.endswith("m"):
            return float(cpu_str[:-1])
        elif cpu_str.replace('.', '', 1).isdigit():
            return float(cpu_str) * 1000
        return 0.0
    except:
        return 0.0

def parse_memory(mem_str: str) -> float:
    try:
        if not mem_str or not isinstance(mem_str, str):
            return 0.0
        mem_str = mem_str.strip().lower().split()[0].split('(')[0]
        if mem_str.endswith("ki"):
            return float(mem_str[:-2]) / 1024
        elif mem_str.endswith("mi"):
            return float(mem_str[:-2])
        elif mem_str.endswith("gi"):
            return float(mem_str[:-2]) * 1024
        return float(mem_str)
    except:
        return 0.0

# Головна функція
def analyze_node(filepath: str):
    print(f"\nАналіз файлу: {os.path.basename(filepath)}")

    with open(filepath, "r", encoding="utf-8") as f:
        lines = [line.rstrip('\n') for line in f]

    node_name = lines[1].split()[-1]
    print(f"Node Name: {node_name}")

    # Capacity
    capacity = {}
    capacity_start = False
    for line in lines:
        if line.startswith("Capacity:"):
            capacity_start = True
            continue
        if capacity_start:
            if line.startswith("Allocatable:"):
                break
            if ":" in line:
                k, v = line.strip().split(":", 1)
                capacity[k.strip()] = v.strip()

    cpu_cap = parse_cpu(capacity.get("cpu", "0"))
    mem_cap = parse_memory(capacity.get("memory", "0"))
    pods_cap = int(capacity.get("pods", "0"))

    # Allocatable
    alloc = {}
    alloc_start = False
    for line in lines:
        if line.startswith("Allocatable:"):
            alloc_start = True
            continue
        if alloc_start:
            if ":" not in line:
                break
            k, v = line.strip().split(":", 1)
            alloc[k.strip()] = v.strip()

    cpu_alloc = parse_cpu(alloc.get("cpu", "0"))
    mem_alloc = parse_memory(alloc.get("memory", "0"))
    pods_alloc = int(alloc.get("pods", "0"))

    # Non-terminated Pods секція
    pods_section = []
    inside = False
    total_pods = None
    for line in lines:
        if "Non-terminated Pods:" in line:
            inside = True
            match = re.search(r"\((\d+)\s+in total\)", line)
            if match:
                total_pods = int(match.group(1))
            continue
        if inside:
            if not line.strip():
                break
            pods_section.append(line.strip())

    header_line = next((l for l in pods_section if l.startswith("Namespace")), None)
    if not header_line:
        print("Таблиця Non-terminated Pods не знайдена.")
        return None

    headers = re.split(r'\s{2,}', header_line)
    data_start = pods_section.index(header_line) + 2
    pod_data = []
    for row in pods_section[data_start:]:
        if re.match(r'^[-\s]+$', row):
            continue
        cols = re.split(r'\s{2,}', row.strip())
        if len(cols) >= len(headers):
            pod_data.append(cols[:len(headers)])

    df = pd.DataFrame(pod_data, columns=headers)
    df["CPU Requests_m"] = df["CPU Requests"].apply(parse_cpu)
    df["CPU Limits_m"] = df["CPU Limits"].apply(parse_cpu)
    df["Memory Requests_Mi"] = df["Memory Requests"].apply(parse_memory)
    df["Memory Limits_Mi"] = df["Memory Limits"].apply(parse_memory)

    # Підсумки ресурсів
    cpu_req = df["CPU Requests_m"].sum()
    cpu_lim = df["CPU Limits_m"].sum()
    mem_req = df["Memory Requests_Mi"].sum()
    mem_lim = df["Memory Limits_Mi"].sum()

    print(f"Capacity: CPU {cpu_cap}m, Memory {mem_cap} Mi, Pods {pods_cap}")
    print(f"Allocatable: CPU {cpu_alloc}m, Memory {mem_alloc} Mi, Pods {pods_alloc}")
    print(f"Non-terminated Pods: {total_pods}")
    print(f"CPU Requests: {cpu_req}m | Limits: {cpu_lim}m")
    print(f"Mem Requests: {mem_req} Mi | Limits: {mem_lim} Mi")

    # Перевірки overcommit
    if cpu_lim > cpu_alloc * 3:
        print("❌ CPU limits > 300% — критичне перевищення! Рекомендується реорганізувати вузли.")
    elif cpu_lim > cpu_alloc:
        print("⚠️ CPU limits > 100% — можливий throttling.")
    else:
        print("✅ CPU limits в нормі.")

    if mem_req > mem_alloc:
        print("❌ Memory requests > allocatable — порушення!")
    elif mem_lim > mem_alloc:
        print("⚠️ Memory limits перевищує allocatable.")
    else:
        print("✅ Memory usage в межах норми.")

    # Поди > 50%
    high_pods = []
    over_100_pods = []
    db_keywords = ["mongo", "redis", "postgresql", "mysql", "couchbase"]
    db_pods = []

    for _, row in df.iterrows():
        cpu_req_ratio = row["CPU Requests_m"] / cpu_alloc * 100 if cpu_alloc else 0
        mem_req_ratio = row["Memory Requests_Mi"] / mem_alloc * 100 if mem_alloc else 0
        
        # Поди > 50%
        if cpu_req_ratio > 50 or mem_req_ratio > 50:
            high_pods.append({
                "Name": row["Name"],
                "CPU %": round(cpu_req_ratio, 2),
                "Mem %": round(mem_req_ratio, 2)
            })
        
        # Поди > 100%
        if cpu_req_ratio > 100 or mem_req_ratio > 100:
            over_100_pods.append({
                "Name": row["Name"],
                "CPU %": round(cpu_req_ratio, 2),
                "Mem %": round(mem_req_ratio, 2)
            })
        
        # Виявлення баз даних
        if any(kw in row["Name"].lower() for kw in db_keywords):
            db_pods.append(row["Name"])

    if high_pods:
        print("\nПоди, які використовують >50% ресурсів:")
        print(pd.DataFrame(high_pods).to_string(index=False))
    else:
        print("\nЖоден pod не перевищує 50% ресурсів.")

    if over_100_pods:
        print("\nПоди, які перевищують 100% ресурсів:")
        print(pd.DataFrame(over_100_pods).to_string(index=False))
    else:
        print("\nЖоден pod не перевищує 100% ресурсів.")

    if db_pods:
        print("\nВиявлені бази даних:")
        for name in db_pods:
            print(f" - {name}")
    else:
        print("\nБази даних не виявлено.")

    # Non-terminated pods — список
    print("\nNon-terminated pods:")
    for name in df["Name"]:
        print(" -", name)

    # Поди без requests
    no_requests = df[(df["CPU Requests_m"] == 0.0) & (df["Memory Requests_Mi"] == 0.0)]["Name"].tolist()
    if no_requests:
        print("\n⚠️ Поди без requests:")
        for name in no_requests:
            print(" -", name)

    # Поди без limits
    no_limits = df[(df["CPU Limits_m"] == 0.0) & (df["Memory Limits_Mi"] == 0.0)]["Name"].tolist()
    if no_limits:
        print("\n⚠️ Поди без limits:")
        for name in no_limits:
            print(" -", name)

    return {
        "name": node_name,
        "df": df,
        "cpu_alloc": cpu_alloc,
        "mem_alloc": mem_alloc,
        "cpu_used": cpu_req,
        "mem_used": mem_req
    }


In [5]:
# node_result_1 = analyze_node(file_map["nodes"][0])


In [6]:
# node_result_2 = analyze_node(file_map["nodes"][1])


# Повторно викликає `analyze_node` для кожного node-файлу, збирає загальні ресурси
та об’єднує усі поди в один DataFrame для подальшого аналізу та перепаковки.

In [7]:
node_results = [analyze_node(p) for p in file_map["nodes"]]

nodes_df = pd.DataFrame([{
        "Node": r["name"],
        "CPU_alloc_m": r["cpu_alloc"],
        "Mem_alloc_Mi": r["mem_alloc"],
        "CPU_used_m": r["cpu_used"],
        "Mem_used_Mi": r["mem_used"]
    } for r in node_results])

cluster_df = pd.concat(
    [r["df"].assign(Node=r["name"]) for r in node_results],
    ignore_index=True
)

print(nodes_df[["Node", "CPU_alloc_m", "Mem_alloc_Mi"]])



Аналіз файлу: 02_describe_node.log
Node Name: ip-100-100-31-113.eu-west-1.compute.internal
Capacity: CPU 4000.0m, Memory 7834.453125 Mi, Pods 58
Allocatable: CPU 3920.0m, Memory 6841.453125 Mi, Pods 58
Non-terminated Pods: 12
CPU Requests: 2185.0m | Limits: 6100.0m
Mem Requests: 2424.0 Mi | Limits: 4864.0 Mi
⚠️ CPU limits > 100% — можливий throttling.
✅ Memory usage в межах норми.

Поди, які використовують >50% ресурсів:
                  Name  CPU %  Mem %
xenia-56b6dcb558-6hdql  51.28  31.81

Жоден pod не перевищує 100% ресурсів.

Бази даних не виявлено.

Non-terminated pods:
 - calico-node-brn8p
 - csi-node-driver-878fn
 - falcon-sensor-k5xzq
 - aws-node-cj5dj
 - aws-node-termination-handler-g98fx
 - ebs-csi-node-hsd8l
 - kube-proxy-ftgcs
 - fluent-bit-kszcv
 - monitoring-prometheus-node-exporter-g9jch
 - compliance-benchmarker-x8s8c
 - fluentd-node-jpch7
 - xenia-56b6dcb558-6hdql

⚠️ Поди без requests:
 - calico-node-brn8p
 - csi-node-driver-878fn
 - falcon-sensor-k5xzq
 - aws-nod

# Реалізує best-fit-decreasing алгоритм перепаковки workload-подів на ноди.
 Обчислює реальне використання ресурсів, переміщення, залишок DaemonSet-ів,
 формує звіт по використанню CPU/RAM і список непоміщених подів.


In [8]:
import re, pandas as pd

# Константи / патерни
MIN_CPU_m  = 50
MIN_MEM_Mi = 64
DAEMON_PATTERNS = [
    r"^kube-proxy-", r"^aws-node-", r"^calico-node-",
    r"^fluent-bit-", r"^fluentd-node-", r"^ebs-csi-node-",
    r"^csi-node-driver-", r"^aws-node-termination-handler-",
    r"^monitoring-prometheus-node-exporter-", r"^compliance-benchmarker-",
    r"^falcon-sensor-"
]
daemon_re = re.compile("|".join(DAEMON_PATTERNS), re.I)

# Нормалізація pod-ов
def normalize_pods(df: pd.DataFrame) -> pd.DataFrame:
    cpu = df[["CPU Requests_m", "CPU Limits_m"]].max(axis=1).clip(lower=MIN_CPU_m)
    mem = df[["Memory Requests_Mi", "Memory Limits_Mi"]].max(axis=1).clip(lower=MIN_MEM_Mi)
    is_daemon = df["Name"].str.match(daemon_re)
    return df.assign(cpu=cpu, mem=mem, is_daemon=is_daemon)

# Best-Fit-Decreasing (только workload)
def best_fit_decreasing(pods_df: pd.DataFrame, nodes_cap: dict):
    placement, unplaced = {}, []
    pods_sorted = pods_df.sort_values(["cpu", "mem"], ascending=False).to_dict("records")

    for pod in pods_sorted:
        if pod["is_daemon"]:
            continue
        cand = [(n, c) for n, c in nodes_cap.items()
                if c["CPU"] >= pod["cpu"] and c["RAM"] >= pod["mem"]]
        if not cand:
            unplaced.append(pod)
            continue
        n, cap = min(cand, key=lambda t: (t[1]["CPU"]-pod["cpu"]) + (t[1]["RAM"]-pod["mem"]))
        placement.setdefault(n, []).append(pod["Name"])
        cap["CPU"] -= pod["cpu"];  cap["RAM"] -= pod["mem"]
    return placement, unplaced

# перепаковка + переміщення
def repack_cluster(pods_raw: pd.DataFrame, nodes_raw: pd.DataFrame):
    pods_df = normalize_pods(pods_raw)

    # повна allocatable ємність
    cap = {r.Node: {"CPU": r.CPU_alloc_m, "RAM": r.Mem_alloc_Mi}
           for r in nodes_raw.itertuples()}

    # резерв DaemonSet-ов
    for _, p in pods_df[pods_df.is_daemon].iterrows():
        cap[p.Node]["CPU"] -= p.cpu
        cap[p.Node]["RAM"] -= p.mem

    # нове розміщення workload
    placement, unplaced = best_fit_decreasing(pods_df, cap)

    # Таблиця по нодам
    rows = []
    for n, r in nodes_raw.set_index("Node").iterrows():
        ds_cpu = pods_df.query("Node == @n and is_daemon")["cpu"].sum()
        ds_ram = pods_df.query("Node == @n and is_daemon")["mem"].sum()
        wl_cpu = pods_df[pods_df.Name.isin(placement.get(n, []))]["cpu"].sum()
        wl_ram = pods_df[pods_df.Name.isin(placement.get(n, []))]["mem"].sum()
        rows.append({
            "Node": n,
            "Alloc CPU m": r.CPU_alloc_m, "Alloc RAM Mi": r.Mem_alloc_Mi,
            "DaemonSet CPU m": ds_cpu,    "DaemonSet RAM Mi": ds_ram,
            "Workload CPU m": wl_cpu,     "Workload RAM Mi": wl_ram,
            "FREE CPU m": cap[n]["CPU"],  "FREE RAM Mi": cap[n]["RAM"],
        })
    rep_df = pd.DataFrame(rows)
    display(rep_df.style.format(thousands=" "))

    # «старе → нове» для всіх pod-ов
    orig_node = pods_df.set_index("Name")["Node"].to_dict()
    new_node  = {
        pod: node for node, lst in placement.items() for pod in lst
    }
    moves = []
    for _, row in pods_df.iterrows():
        name  = row["Name"]
        old   = orig_node[name]
        # DaemonSet-ы лишаються там же
        new   = new_node.get(name, old if row["is_daemon"] else None)
        status = ("stay" if new == old else
                  "move" if new else
                  "unplaced")
        moves.append({
            "Pod": name, "Old node": old, "New node": new, "Status": status,
            "CPU m": row["cpu"], "RAM Mi": row["mem"]
        })
    moves_df = pd.DataFrame(moves).sort_values(["Status","Pod"])
    print("\n📦 Рух pod-ов (stay / move / unplaced):")
    display(moves_df)

    # список «не умістились»
    if unplaced:
        print("\n❌ Pod-и, не уміщаються в жодну ноду:")
        max_cpu = max(c["CPU"] for c in cap.values())
        max_ram = max(c["RAM"] for c in cap.values())
        for p in unplaced:
            print(f" • {p['Name']:<28}  CPU {p['cpu']}m  (+{p['cpu']-max_cpu}m)  |"
                  f" RAM {p['mem']}Mi  (+{p['mem']-max_ram:.1f}Mi)")
    else:
        print("\n✅ Усі workload-поди уміщаються.")

    # Ітоговий дефіцит
    def_cpu = max(0, sum(p["cpu"] for p in unplaced) - sum(c["CPU"] for c in cap.values()))
    def_ram = max(0, sum(p["mem"] for p in unplaced) - sum(c["RAM"] for c in cap.values()))
    print(f"\nΔ  Дефіцит: CPU {def_cpu}m  |  RAM {def_ram}Mi")

    return placement, unplaced, rep_df, moves_df

# Запуск
placement, unplaced, rep_df, moves_df = repack_cluster(cluster_df, nodes_df)


# тест на випадкових данних
def demo_scenario():
    # 2 ноди по 2000 mCPU / 4 GiB
    demo_nodes = pd.DataFrame([
        {"Node": "node-A", "CPU_alloc_m": 2000, "Mem_alloc_Mi": 4096},
        {"Node": "node-B", "CPU_alloc_m": 2000, "Mem_alloc_Mi": 4096},
    ])

    # 4 workload-пода и 1 DaemonSet (64 m / 64 Mi на кожній ноді)
    demo_pods = pd.DataFrame([
        {"Name": "ds-logger-A", "Node": "node-A", "CPU Requests_m": 0, "CPU Limits_m": 0,
         "Memory Requests_Mi": 0, "Memory Limits_Mi": 0},                # DaemonSet
        {"Name": "ds-logger-B", "Node": "node-B", "CPU Requests_m": 0, "CPU Limits_m": 0,
         "Memory Requests_Mi": 0, "Memory Limits_Mi": 0},
        # workload: один «товстий» та три «тонких»
        {"Name": "svc-big",  "Node": "node-A", "CPU Requests_m": 1500, "CPU Limits_m": 1500,
         "Memory Requests_Mi": 1024, "Memory Limits_Mi": 1024},
        {"Name": "svc-sml1", "Node": "node-A", "CPU Requests_m": 300,  "CPU Limits_m": 300,
         "Memory Requests_Mi": 300,  "Memory Limits_Mi": 300},
        {"Name": "svc-sml2", "Node": "node-B", "CPU Requests_m": 300,  "CPU Limits_m": 300,
         "Memory Requests_Mi": 300,  "Memory Limits_Mi": 300},
        {"Name": "svc-sml3", "Node": "node-B", "CPU Requests_m": 300,  "CPU Limits_m": 300,
         "Memory Requests_Mi": 300,  "Memory Limits_Mi": 300},
    ])

    print("\n\nDEMO-SCENARIO")
    placement, unplaced, rep_df, moves_df = repack_cluster(demo_pods, demo_nodes)
    return rep_df, moves_df

demo_rep, demo_moves = demo_scenario()


,Node,Alloc CPU m,Alloc RAM Mi,DaemonSet CPU m,DaemonSet RAM Mi,Workload CPU m,Workload RAM Mi,FREE CPU m,FREE RAM Mi
0,ip-100-100-31-113.eu-west-1.compute.internal,3 920.000000,6 841.453125,2 650.000000,2 368.000000,0.000000,0.000000,1 270.000000,4 473.453125
1,ip-100-100-41-106.eu-west-1.compute.internal,1 930.000000,7 120.933594,2 650.000000,2 368.000000,0.000000,0.000000,-720.000000,4 752.933594



📦 Рух pod-ов (stay / move / unplaced):


,Pod,Old node,New node,Status,CPU m,RAM Mi
3,aws-node-cj5dj,ip-100-100-31-113.eu-west-1.compute.internal,ip-100-100-31-113.eu-west-1.compute.internal,stay,50.0,64.0
15,aws-node-fv645,ip-100-100-41-106.eu-west-1.compute.internal,ip-100-100-41-106.eu-west-1.compute.internal,stay,50.0,64.0
4,aws-node-termination-handler-g98fx,ip-100-100-31-113.eu-west-1.compute.internal,ip-100-100-31-113.eu-west-1.compute.internal,stay,50.0,64.0
16,aws-node-termination-handler-gr6wc,ip-100-100-41-106.eu-west-1.compute.internal,ip-100-100-41-106.eu-west-1.compute.internal,stay,50.0,64.0
0,calico-node-brn8p,ip-100-100-31-113.eu-west-1.compute.internal,ip-100-100-31-113.eu-west-1.compute.internal,stay,50.0,64.0
12,calico-node-prf59,ip-100-100-41-106.eu-west-1.compute.internal,ip-100-100-41-106.eu-west-1.compute.internal,stay,50.0,64.0
21,compliance-benchmarker-wn9pz,ip-100-100-41-106.eu-west-1.compute.internal,ip-100-100-41-106.eu-west-1.compute.internal,stay,50.0,64.0
9,compliance-benchmarker-x8s8c,ip-100-100-31-113.eu-west-1.compute.internal,ip-100-100-31-113.eu-west-1.compute.internal,stay,50.0,64.0
1,csi-node-driver-878fn,ip-100-100-31-113.eu-west-1.compute.internal,ip-100-100-31-113.eu-west-1.compute.internal,stay,50.0,64.0
13,csi-node-driver-bnw7b,ip-100-100-41-106.eu-west-1.compute.internal,ip-100-100-41-106.eu-west-1.compute.internal,stay,50.0,64.0



❌ Pod-и, не уміщаються в жодну ноду:
 • xenia-stg-mongodb-0           CPU 4000.0m  (+2730.0m)  | RAM 7168.0Mi  (+2415.1Mi)
 • xenia-56b6dcb558-6hdql        CPU 4000.0m  (+2730.0m)  | RAM 3072.0Mi  (+-1680.9Mi)

Δ  Дефіцит: CPU 7450.0m  |  RAM 1013.61328125Mi


DEMO-SCENARIO


,Node,Alloc CPU m,Alloc RAM Mi,DaemonSet CPU m,DaemonSet RAM Mi,Workload CPU m,Workload RAM Mi,FREE CPU m,FREE RAM Mi
0,node-A,2 000,4 096,0,0,1 900,1 452,100,2 644
1,node-B,2 000,4 096,0,0,600,600,1 400,3 496



📦 Рух pod-ов (stay / move / unplaced):


,Pod,Old node,New node,Status,CPU m,RAM Mi
1,ds-logger-B,node-B,node-A,move,50,64
0,ds-logger-A,node-A,node-A,stay,50,64
2,svc-big,node-A,node-A,stay,1500,1024
3,svc-sml1,node-A,node-A,stay,300,300
4,svc-sml2,node-B,node-B,stay,300,300
5,svc-sml3,node-B,node-B,stay,300,300



✅ Усі workload-поди уміщаються.

Δ  Дефіцит: CPU 0m  |  RAM 0Mi


# Розраховує загальний дефіцит CPU та RAM у кластері на основі непоміщених pod-ів
 та залишкових ресурсів доступних нод. Повертає значення в mCPU та MiB.


In [9]:
# Підсумковий дефіцит CPU / RAM
def summarize_deficit(unplaced, nodes_cap):
    """Повертає словник з total_deficit_cpu_m / mem_mi."""
    total_free_cpu = sum(c['CPU'] for c in nodes_cap.values())
    total_free_ram = sum(c['RAM'] for c in nodes_cap.values())
    need_cpu_m = sum(p['cpu'] for p in unplaced)
    need_ram_mi = sum(p['mem'] for p in unplaced)
    # якщо ресурсів не вистачає – рахуємо хвіст; інакше 0
    deficit_cpu_m = max(0, need_cpu_m - total_free_cpu)
    deficit_ram_mi = max(0, need_ram_mi - total_free_ram)
    return {"cpu_m": deficit_cpu_m, "ram_mi": deficit_ram_mi}


In [10]:
import requests

url = "https://instances.vantage.sh/instances.json"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print(f"Отримано {len(data)} інстансів")
else:
    print(f"Помилка: {response.status_code}")


Отримано 940 інстансів


# Завантажує каталог AWS-інстансів з Vantage, фільтрує інстанси,
які задовольняють дефіцит CPU/RAM, і виводить рекомендації:

-оптимальний за розміром

-найдешевший варіант.


In [11]:
# Рекомендація Worker-ноди (розмір / вартість)
import json, requests, pandas as pd

CATALOG_URL = "https://instances.vantage.sh/instances.json"

def load_vantage_catalog(region="eu-west-1"):
    """Завантажує масив JSON і повертає DataFrame з потрібними полями."""
    data = requests.get(CATALOG_URL, timeout=15).json()        # ← масив ≈ 900 записів
    rows = []
    for inst in data:
        # є ціна для нашого регіону та Linux-OnDemand?
        pr_region = inst["pricing"].get(region, {}).get("linux", {})
        price_str = pr_region.get("ondemand")
        if price_str is None:
            continue
        rows.append({
            "instance_type": inst["instance_type"],
            "vCPU":          int(inst["vCPU"]),
            "memory_GiB":    float(inst["memory"]),            # поле вже в GiB
            "price":         float(price_str),                 # $/hour
        })
    return pd.DataFrame(rows)

def recommend_instances_vantage(deficit, region="eu-west-1"):
    df = load_vantage_catalog(region)
    need_cpu = -(-deficit["cpu_m"]  // 1000)       # ceil → vCPU
    need_ram = -(-deficit["ram_mi"] // 1024)       # ceil → GiB

    candidates = df[(df["vCPU"] >= need_cpu) & (df["memory_GiB"] >= need_ram)]
    if candidates.empty:
        print("❗️  У каталозі немає інстансів, що покривають дефіцит.")
        return

    # мінімальний за vCPU → RAM → $ (окремо по розміру)
    by_size = (candidates
               .sort_values(["vCPU", "memory_GiB", "price"])
               .iloc[0])

    # мінімальний за ціною (окремо по коштах)
    by_cost = candidates.sort_values("price").iloc[0]

    print(f"Дефіцит потрібно покрити ≥ {need_cpu} vCPU / "
          f"≥ {need_ram} GiB (регіон {region})")
    print("Рекомендації Worker Node:")
    print(f"   ▸ За розміром : {by_size.instance_type:<12}  "
          f"{by_size.vCPU} vCPU / {by_size.memory_GiB:.1f} GiB   "
          f"≈ ${by_size.price}/год")
    print(f"   ▸ За ціною   : {by_cost.instance_type:<12}  "
          f"{by_cost.vCPU} vCPU / {by_cost.memory_GiB:.1f} GiB   "
          f"≈ ${by_cost.price}/год")

    return by_size, by_cost, candidates


# Повний пайплайн:

In [12]:
import pandas as pd

def full_analysis(log_dir="all_logs", region="eu-west-1"):
    file_map = load_all_logs(log_dir)

    if file_map["namespace"]:
        print("\n═════════  NAMESPACE SUMMARY  ═════════")
        analyze_namespace(file_map["namespace"])

    print("\n═════════  NODE PARSE  ═════════")
    node_results = [analyze_node(p) for p in file_map["nodes"]]

    nodes_df = pd.DataFrame([{
        "Node": r["name"],
        "CPU_alloc_m":  r["cpu_alloc"],
        "Mem_alloc_Mi": r["mem_alloc"],
        "CPU_used_m":   r["cpu_used"],
        "Mem_used_Mi":  r["mem_used"],
    } for r in node_results])

    cluster_df = pd.concat(
        [r["df"].assign(Node=r["name"]) for r in node_results],
        ignore_index=True
    )

    print("\n═════════  REPACK WORKLOAD  ═════════")
    placement, unplaced, rep_df, moves_df = repack_cluster(cluster_df, nodes_df)

    nodes_cap = {                          
        r["Node"]: {"CPU": r["FREE CPU m"], "RAM": r["FREE RAM Mi"]}
        for _, r in rep_df.iterrows()
    }
    
    deficit = summarize_deficit(unplaced, nodes_cap)
    print("Дефіцит кластера:")
    print(f"   CPU : {deficit['cpu_m']/1000:.2f} vCPU  ({deficit['cpu_m']} m)")
    print(f"   RAM : {deficit['ram_mi']/1024:.2f} GiB  ({deficit['ram_mi']} Mi)")


    print("\n═════════  INSTANCE RECOMMENDATION  ═════════")
    by_size, by_cost, _ = recommend_instances_vantage(deficit, region=region)

    return {
        "file_map":     file_map,
        "nodes_df":     nodes_df,
        "cluster_df":   cluster_df,
        "repack_table": rep_df,
        "moves_df":     moves_df,
        "unplaced":     unplaced,
        "deficit":      deficit,
        "by_size":      by_size,
        "by_cost":      by_cost,
    }


result = full_analysis(log_dir="all_logs", region="eu-west-1")



═════════  NAMESPACE SUMMARY  ═════════

Аналіз namespace з файлу: all_logs\01_namespace_sample.log
Завантажено 10 pod-ів

Статус подів:
STATUS
Completed    5
Running      5
Name: count, dtype: int64

Поди по нодах:
NODE
ip-100-100-24-129.eu-west-1.compute.internal    4
ip-100-100-46-4.eu-west-1.compute.internal      1
ip-100-100-31-113.eu-west-1.compute.internal    1
ip-100-100-35-194.eu-west-1.compute.internal    1
ip-100-100-41-106.eu-west-1.compute.internal    1
ip-100-100-58-211.eu-west-1.compute.internal    1
ip-100-100-17-161.eu-west-1.compute.internal    1
Name: count, dtype: int64

═════════  NODE PARSE  ═════════

Аналіз файлу: 02_describe_node.log
Node Name: ip-100-100-31-113.eu-west-1.compute.internal
Capacity: CPU 4000.0m, Memory 7834.453125 Mi, Pods 58
Allocatable: CPU 3920.0m, Memory 6841.453125 Mi, Pods 58
Non-terminated Pods: 12
CPU Requests: 2185.0m | Limits: 6100.0m
Mem Requests: 2424.0 Mi | Limits: 4864.0 Mi
⚠️ CPU limits > 100% — можливий throttling.
✅ Memory usag

,Node,Alloc CPU m,Alloc RAM Mi,DaemonSet CPU m,DaemonSet RAM Mi,Workload CPU m,Workload RAM Mi,FREE CPU m,FREE RAM Mi
0,ip-100-100-31-113.eu-west-1.compute.internal,3 920.000000,6 841.453125,2 650.000000,2 368.000000,0.000000,0.000000,1 270.000000,4 473.453125
1,ip-100-100-41-106.eu-west-1.compute.internal,1 930.000000,7 120.933594,2 650.000000,2 368.000000,0.000000,0.000000,-720.000000,4 752.933594



📦 Рух pod-ов (stay / move / unplaced):


,Pod,Old node,New node,Status,CPU m,RAM Mi
3,aws-node-cj5dj,ip-100-100-31-113.eu-west-1.compute.internal,ip-100-100-31-113.eu-west-1.compute.internal,stay,50.0,64.0
15,aws-node-fv645,ip-100-100-41-106.eu-west-1.compute.internal,ip-100-100-41-106.eu-west-1.compute.internal,stay,50.0,64.0
4,aws-node-termination-handler-g98fx,ip-100-100-31-113.eu-west-1.compute.internal,ip-100-100-31-113.eu-west-1.compute.internal,stay,50.0,64.0
16,aws-node-termination-handler-gr6wc,ip-100-100-41-106.eu-west-1.compute.internal,ip-100-100-41-106.eu-west-1.compute.internal,stay,50.0,64.0
0,calico-node-brn8p,ip-100-100-31-113.eu-west-1.compute.internal,ip-100-100-31-113.eu-west-1.compute.internal,stay,50.0,64.0
12,calico-node-prf59,ip-100-100-41-106.eu-west-1.compute.internal,ip-100-100-41-106.eu-west-1.compute.internal,stay,50.0,64.0
21,compliance-benchmarker-wn9pz,ip-100-100-41-106.eu-west-1.compute.internal,ip-100-100-41-106.eu-west-1.compute.internal,stay,50.0,64.0
9,compliance-benchmarker-x8s8c,ip-100-100-31-113.eu-west-1.compute.internal,ip-100-100-31-113.eu-west-1.compute.internal,stay,50.0,64.0
1,csi-node-driver-878fn,ip-100-100-31-113.eu-west-1.compute.internal,ip-100-100-31-113.eu-west-1.compute.internal,stay,50.0,64.0
13,csi-node-driver-bnw7b,ip-100-100-41-106.eu-west-1.compute.internal,ip-100-100-41-106.eu-west-1.compute.internal,stay,50.0,64.0



❌ Pod-и, не уміщаються в жодну ноду:
 • xenia-stg-mongodb-0           CPU 4000.0m  (+2730.0m)  | RAM 7168.0Mi  (+2415.1Mi)
 • xenia-56b6dcb558-6hdql        CPU 4000.0m  (+2730.0m)  | RAM 3072.0Mi  (+-1680.9Mi)

Δ  Дефіцит: CPU 7450.0m  |  RAM 1013.61328125Mi
Дефіцит кластера:
   CPU : 7.45 vCPU  (7450.0 m)
   RAM : 0.99 GiB  (1013.61328125 Mi)

═════════  INSTANCE RECOMMENDATION  ═════════
Дефіцит потрібно покрити ≥ 8.0 vCPU / ≥ 1.0 GiB (регіон eu-west-1)
Рекомендації Worker Node:
   ▸ За розміром : c1.xlarge     8 vCPU / 7.0 GiB   ≈ $0.592/год
   ▸ За ціною   : a1.2xlarge    8 vCPU / 16.0 GiB   ≈ $0.2304/год
